## 7: AgentCore Gateway Policies (OPTIONAL)

## Overview

This optional lab demonstrates how to add fine-grained authorization to your AgentCore Gateway using Cedar policies. You'll learn to restrict tool access based on conditions, implementing principle of least privilege.

## What You'll Learn

- Create an AgentCore Policy Engine
- Write Cedar policies for conditional access control
- Attach policies to enforce authorization rules
- Test policy enforcement

## Prerequisites

- Completed 2 (AgentCore Identity)
- Completed 3 (AgentCore Gateway)
- Gateway must be created and accessible

## Why Use Gateway Policies?

Without policies, any authenticated user can access all gateway tools. Policies enable:
- Conditional authorization based on request parameters
- Role-based access control
- Fine-grained permission management
- Audit and compliance tracking

### Step 1: Install/Upgrade Toolkit with Policy Support

The Policy module requires the latest version of the AgentCore toolkit. Run this cell to upgrade.

In [ ]:
# Upgrade to the latest version of the toolkit
!pip install --upgrade bedrock-agentcore-starter-toolkit --quiet

print("✅ Toolkit upgraded")
print("\n⚠️  IMPORTANT: After upgrading, restart the kernel:")
print("   1. Click 'Kernel' menu")
print("   2. Select 'Restart Kernel'")
print("   3. Then re-run all cells from the beginning")

### Step 2: Verify Policy Module Installation

In [ ]:
# Check if policy module is available
try:
    from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient
    print("✅ Policy module is available")
    print("   You can proceed with this lab")
except ImportError as e:
    print("❌ Policy module is NOT available")
    print(f"   Error: {e}")
    print("\n📝 Troubleshooting:")
    print("   1. Make sure you ran the upgrade cell above")
    print("   2. Restart the kernel (Kernel > Restart Kernel)")
    print("   3. Re-run all cells from the beginning")
    print("   4. If still not working, this lab is optional - skip to 8")

## Load Configuration

Load the gateway and Cognito configuration from previous labs.

In [ ]:
import os
import json
import boto3
import time
from utils.identity_ssm_utils import reauthenticate_user

# Initialize AWS session
region = boto3.Session().region_name or 'us-west-2'

# Load Cognito configuration from 2
with open('cognito_config.json', 'r') as f:
    cognito_config = json.load(f)
cognito_client_id = cognito_config['client_id']

# Load gateway configuration from 3
with open('gateway_config.json', 'r') as f:
    gateway = json.load(f)

print(f"✅ Configuration loaded")
print(f"   Gateway ID: {gateway['id']}")
print(f"   Region: {region}")

## Create Policy Engine

Create a Policy Engine to contain Cedar authorization policies for fine-grained access control.

In [ ]:
# Try to import from toolkit, fall back to custom implementation if not available
try:
    from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient
    print("✅ Using toolkit PolicyClient")
except ImportError:
    from utils.policy_utils import PolicyClient
    print("✅ Using custom PolicyClient (toolkit policy module not available)")

# Initialize the policy client
policy_client = PolicyClient(region_name=region)

print("\n🔧 Creating Policy Engine...")

# Create or get existing policy engine
# The policy engine is a container for all our authorization policies
engine = policy_client.create_or_get_policy_engine(
    name="RefundAgentPolicy",
    description="Policy engine for refund processing governance",
)

engine_id = engine['policyEngineId']
engine_arn = engine['policyEngineArn']

print(f"\n✅ Policy Engine ready")
print(f"   Engine ID: {engine_id}")
print(f"   Engine ARN: {engine_arn}")

## Create Cedar Policy

Generate Cedar policies to enforce scope-based access: write scope for approve operations, read scope for create/list operations.

In [ ]:
# Create Cedar policy for approve tool (write scope)
print("\n📝 Generating Cedar Policy from Natural language...")

nl_input = "Only users with scope of 'workshop-api/write' should be allowed to access approve refund tool."

approve_result = policy_client.generate_policy(
    policy_engine_id=engine["policyEngineId"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": gateway["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,
)

# Create Cedar policy for create/list tools (read scope)
nl_input = "Users with scope of 'workshop-api/read' should be allowed to access ListReturnRequest and CreateRefundRequest tools."

create_result = policy_client.generate_policy(
    policy_engine_id=engine["policyEngineId"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": gateway["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,
)

print("✅ Policies generated from natural language")

In [ ]:
print("📋 Generated Cedar Policies:\n")
print("=" * 80)

# Display the approve policy
print("\n1️⃣  APPROVE REFUND POLICY (write scope required):")
print("-" * 80)
cedar_policy_approve = approve_result["generatedPolicies"][0]["definition"]["cedar"]["statement"]
print(cedar_policy_approve)

print("\n" + "=" * 80)

# Display the create/list policy
print("\n2️⃣  CREATE/LIST REFUND POLICY (read scope required):")
print("-" * 80)
cedar_policy_read = create_result["generatedPolicies"][0]["definition"]["cedar"]["statement"]
print(cedar_policy_read)

print("\n" + "=" * 80)

## Create Policies in Policy Engine

Create the generated Cedar policies in the Policy Engine.

In [ ]:
print("🔧 Creating policies in Policy Engine...\n")

# Create the approve policy
if approve_result.get("status") == "GENERATED" and approve_result.get("generatedPolicies"):
    generated_policy = approve_result["generatedPolicies"][0]
    approve_policy = policy_client.create_or_get_policy(
        policy_engine_id=engine["policyEngineId"],
        name="approve_tool_policy",
        description="Allow approve tool call only with write scope",
        definition=generated_policy.get("definition", {}),
    )
    print("✅ Policy ready: approve_tool_policy")
    print("   Scope required: workshop-api/write")
    print("   Tools allowed: ApproveReturnRequest\n")

# Create the create/list policy
if create_result.get("status") == "GENERATED" and create_result.get("generatedPolicies"):
    generated_policy = create_result["generatedPolicies"][0]
    read_policy = policy_client.create_or_get_policy(
        policy_engine_id=engine["policyEngineId"],
        name="create_list_tool_policy",
        description="Allow create and list tool calls only with read scope",
        definition=generated_policy.get("definition", {}),
    )
    print("✅ Policy ready: create_list_tool_policy")
    print("   Scope required: workshop-api/read")
    print("   Tools allowed: CreateRefundRequest, ListReturnRequest\n")

print("✅ All policies ready!")

## Update Gateway IAM Role Permissions

Before attaching the Policy Engine, we need to grant the Gateway IAM role permission to access the Policy Engine.

In [ ]:
import boto3
import json

# Get account ID and region
session = boto3.Session()
sts = session.client('sts')
account_id = sts.get_caller_identity()['Account']
region = session.region_name or 'us-west-2'

iam_client = boto3.client('iam')
role_name = "RefundManagementGatewayExecutionRole"

print("🔧 Updating Gateway IAM role with Policy Engine permissions...")

# Policy document that grants access to Policy Engine
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:*"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region}:{account_id}:policy-engine/*",
                f"arn:aws:bedrock-agentcore:{region}:{account_id}:gateway/*"
            ]
        }
    ]
}

try:
    # Add inline policy to the role
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="PolicyEngineAccess",
        PolicyDocument=json.dumps(policy_document)
    )
    
    print(f"✅ IAM role updated successfully")
    print(f"   Role: {role_name}")
    print(f"   Added permissions: GetPolicyEngine, GetPolicy, ListPolicies")
    print("\n⏳ Waiting 10 seconds for IAM changes to propagate...")
    
    import time
    time.sleep(10)
    
    print("✅ Ready to attach Policy Engine")
    
except Exception as e:
    print(f"❌ Error updating IAM role: {e}")
    print("\nYou may need to manually add these permissions to the role.")

## Attach Policy Engine to Gateway

Enable policy enforcement by attaching the Policy Engine to the Gateway in ENFORCE mode.

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

# Initialize gateway client
gateway_client_toolkit = GatewayClient(region_name=region)

print("🔧 Attaching Policy Engine to Gateway...")
print(f"   Mode: ENFORCE (policies will block unauthorized requests)\n")

# Attach the policy engine to the gateway
update_response = gateway_client_toolkit.update_gateway_policy_engine(
    gateway_identifier=gateway["id"],
    policy_engine_arn=engine["policyEngineArn"],
    mode="ENFORCE"
)

print("✅ Policy Engine attached successfully!")
print(f"   Gateway ID: {gateway['id']}")
print(f"   Policy Engine: {engine['policyEngineId']}")
print(f"   Mode: ENFORCE")
print("\n🔒 Authorization is now active!")

## Test Policy Enforcement - Write Scope

Test with write scope token - should only see approve tool.

In [ ]:
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from utils.identity_ssm_utils import get_cognito_token_with_scope

print("🧪 Testing Policy Enforcement - Write Scope")
print("=" * 80)

# Get OAuth token with WRITE scope
print("\n1️⃣  Requesting token with scope: workshop-api/write")
try:
    token_write = get_cognito_token_with_scope(
        cognito_config['client_id'],
        cognito_config['client_secret'],
        cognito_config['discovery_url'],
        "workshop-api/write"
    )
    print("   ✓ Token obtained\n")
except Exception as e:
    print(f"   ❌ Error getting token: {e}")
    print("\n📝 Troubleshooting:")
    print("   1. Verify Cognito resource server has 'workshop-api/write' scope")
    print("   2. Check that the app client has this scope enabled")
    print("   3. Ensure the app client has client_credentials flow enabled")
    raise

# Create MCP client with write scope token
print("2️⃣  Listing available tools with write scope token:")
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway["gateway_url"],
        headers={"Authorization": f"Bearer {token_write}"},
    )
)

try:
    # List tools - should only see approve tool
    with mcp_client:
        tools = list(mcp_client.list_tools_sync())
        print(f"   Found {len(tools)} tool(s):\n")
        for tool in tools:
            print(f"   ✅ {tool.mcp_tool.name}")
            print(f"      {tool.mcp_tool.description}\n")
    
    print("=" * 80)
    print("\n✅ Write scope test complete!")
    print("   Expected: Only approve_refund_request tool")
    print("   Actual: See tools listed above")
    
except Exception as e:
    print(f"\n❌ Error listing tools: {e}")
    print("\n📝 Troubleshooting:")
    print("   1. Verify the Policy Engine is attached to the Gateway")
    print("   2. Check that policies were created successfully")
    print("   3. Ensure the token has the correct scope claim")
    print("   4. Check CloudWatch logs for policy evaluation details")
    print("   5. Verify Gateway IAM role has Policy Engine permissions")

## Test Policy Enforcement - Read Scope

Test with read scope token - should see create and list tools, but NOT approve.

In [ ]:
print("🧪 Testing Policy Enforcement - Read Scope")
print("=" * 80)

# Get OAuth token with READ scope
print("\n1️⃣  Requesting token with scope: workshop-api/read")
try:
    token_read = get_cognito_token_with_scope(
        cognito_config['client_id'],
        cognito_config['client_secret'],
        cognito_config['discovery_url'],
        "workshop-api/read"
    )
    print("   ✓ Token obtained\n")
except Exception as e:
    print(f"   ❌ Error getting token: {e}")
    print("\n📝 Troubleshooting:")
    print("   1. Verify Cognito resource server has 'workshop-api/read' scope")
    print("   2. Check that the app client has this scope enabled")
    print("   3. Ensure the app client has client_credentials flow enabled")
    raise

# Create MCP client with read scope token
print("2️⃣  Listing available tools with read scope token:")
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway["gateway_url"],
        headers={"Authorization": f"Bearer {token_read}"},
    )
)

try:
    # List tools - should see create and list tools, but NOT approve
    with mcp_client:
        tools = list(mcp_client.list_tools_sync())
        print(f"   Found {len(tools)} tool(s):\n")
        for tool in tools:
            print(f"   ✅ {tool.mcp_tool.name}")
            print(f"      {tool.mcp_tool.description}\n")
    
    print("=" * 80)
    print("\n✅ Read scope test complete!")
    print("   Expected: create_refund_request and list_refund_requests tools")
    print("   Actual: See tools listed above")
    
except Exception as e:
    print(f"\n❌ Error listing tools: {e}")
    print("\n📝 Troubleshooting:")
    print("   1. Verify the Policy Engine is attached to the Gateway")
    print("   2. Check that policies were created successfully")
    print("   3. Ensure the token has the correct scope claim")
    print("   4. Check CloudWatch logs for policy evaluation details")
    print("   5. Verify Gateway IAM role has Policy Engine permissions")

## Summary

You've successfully implemented fine-grained authorization using Cedar policies:

- Created a Policy Engine to contain authorization rules
- Generated Cedar policies from natural language descriptions
- Attached policies to enforce scope-based access control
- Tested policy enforcement with different OAuth scopes

**Key Takeaways:**
- Write scope (`workshop-api/write`) grants access to approve tool only
- Read scope (`workshop-api/read`) grants access to create and list tools only
- Cedar policies provide fine-grained, declarative authorization
- Policy enforcement happens at the gateway level, outside application code

## Next Steps

- **8: Streamlit Frontend (OPTIONAL)** - Build a customer-facing application
- **9: Cleanup Resources** - Remove all resources created in this workshop